In [27]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
import pytz

In [28]:
start_time = datetime(2014, 12, 31, 23, 59, 0)

In [29]:
COMPANIES_FILE_PATH = 'nz_main_board_companies.csv'
STOCK_FILE_PATH = 'nz_main_stock_data.csv'
START_TIME = pytz.timezone('Pacific/Auckland').localize((start_time))
KEY_COLS = ['close','dividends', 'company', 'sector', 'industry']
PERIOD = '10y'


In [30]:
nz_main_board_companies = pd.read_csv(COMPANIES_FILE_PATH)
nz_main_board_companies.head()

,Code,Company
0,WIN,Winton Land Limited (NS)
1,WHS,The Warehouse Group Limited
2,WCO,WasteCo Group Limited
3,WBC,Westpac Banking Corporation
4,VTL,Vital Limited


In [31]:
def get_stock_data(df: pd.DataFrame) -> pd.DataFrame:
    stock_df = pd.DataFrame()
    for _, (code, company) in df.iterrows():
        try:
            ticker = yf.Ticker(f"{code}.NZ")
            cur_stock_df = ticker.history(period=PERIOD)
            info = ticker.info
            if cur_stock_df.empty:
                continue
            cur_stock_df['company'] = company
            cur_stock_df['sector'] = info.get('sector')
            cur_stock_df['industry'] = info.get('industry')
            stock_df = pd.concat([stock_df, cur_stock_df])
        except KeyError as e:
            print(f"{company} data is not found in Yahoo Finance!")
        except Exception as e:
            print(f"Error fetching data for {company}: {e}")
    return stock_df 

In [32]:
try:
    stock_df = pd.read_csv(STOCK_FILE_PATH, index_col='Date')
except FileNotFoundError:
    stock_df = get_stock_data(nz_main_board_companies)
    stock_df.to_csv(STOCK_FILE_PATH)

stock_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,company,sector,industry,Capital Gains
Date,,,,,,,,,,,
2021-12-17 00:00:00+13:00,3.835612,3.835612,3.728526,3.767467,292826,0.0,0.0,Winton Land Limited (NS),Real Estate,Real Estate - Development,NaN
2021-12-20 00:00:00+13:00,3.767466,3.767466,3.611706,3.621441,34590,0.0,0.0,Winton Land Limited (NS),Real Estate,Real Estate - Development,NaN
2021-12-21 00:00:00+13:00,3.699321,3.699321,3.699321,3.699321,18354,0.0,0.0,Winton Land Limited (NS),Real Estate,Real Estate - Development,NaN
2021-12-22 00:00:00+13:00,3.689586,3.689586,3.640911,3.689586,14486,0.0,0.0,Winton Land Limited (NS),Real Estate,Real Estate - Development,NaN
2021-12-23 00:00:00+13:00,3.631176,3.660381,3.631176,3.650646,10024,0.0,0.0,Winton Land Limited (NS),Real Estate,Real Estate - Development,NaN


In [33]:
stock_df.columns = stock_df.columns.str.lower()
stock_df = stock_df[KEY_COLS]
stock_df['close_ffill'] = stock_df['close'].ffill()
dividends_df = stock_df[stock_df['dividends'] > 0].copy()
dividends_df['close'] = dividends_df['close'].combine_first(dividends_df['close_ffill'])
dividends_df.drop('close_ffill', axis=1, inplace=True)
dividends_df.index = pd.to_datetime(dividends_df.index, utc=True).tz_convert('Pacific/Auckland')
dividends_df['year'] = dividends_df.index.year
dividends_df

,close,dividends,company,sector,industry,year
Date,,,,,,
2022-08-30 00:00:00+12:00,2.541574,0.010700,Winton Land Limited (NS),Real Estate,Real Estate - Development,2022
2023-02-28 00:00:00+13:00,1.916445,0.020600,Winton Land Limited (NS),Real Estate,Real Estate - Development,2023
2023-08-28 00:00:00+12:00,2.342819,0.021600,Winton Land Limited (NS),Real Estate,Real Estate - Development,2023
2024-02-26 00:00:00+13:00,2.500000,0.007639,Winton Land Limited (NS),Real Estate,Real Estate - Development,2024
2015-11-25 00:00:00+13:00,1.487685,0.050000,The Warehouse Group Limited,Consumer Cyclical,Department Stores,2015
...,...,...,...,...,...,...
2024-08-14 00:00:00+12:00,6.808300,0.157600,Australian Foundation Investment Company Limited,Financial Services,Asset Management,2024
2025-02-03 00:00:00+13:00,6.940000,0.131700,Australian Foundation Investment Company Limited,Financial Services,Asset Management,2025
2023-11-21 00:00:00+13:00,0.769250,0.041600,2 Cheap Cars Group Limited,Consumer Cyclical,Auto & Truck Dealerships,2023


In [34]:
dividends_df.index

DatetimeIndex(['2022-08-30 00:00:00+12:00', '2023-02-28 00:00:00+13:00',
               '2023-08-28 00:00:00+12:00', '2024-02-26 00:00:00+13:00',
               '2015-11-25 00:00:00+13:00', '2016-04-01 00:00:00+13:00',
               '2016-11-24 00:00:00+13:00', '2017-03-31 00:00:00+13:00',
               '2017-11-23 00:00:00+13:00', '2018-03-29 00:00:00+13:00',
               ...
               '2022-02-09 00:00:00+13:00', '2022-08-10 00:00:00+12:00',
               '2023-02-02 00:00:00+13:00', '2023-08-11 00:00:00+12:00',
               '2024-02-02 00:00:00+13:00', '2024-08-14 00:00:00+12:00',
               '2025-02-03 00:00:00+13:00', '2023-11-21 00:00:00+13:00',
               '2024-05-30 00:00:00+12:00', '2024-11-21 00:00:00+13:00'],
              dtype='datetime64[ns, Pacific/Auckland]', name='Date', length=2343, freq=None)

In [35]:
dividends_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2343 entries, 2022-08-30 00:00:00+12:00 to 2024-11-21 00:00:00+13:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   close      2343 non-null   float64
 1   dividends  2343 non-null   float64
 2   company    2343 non-null   object 
 3   sector     1452 non-null   object 
 4   industry   1452 non-null   object 
 5   year       2343 non-null   int32  
dtypes: float64(2), int32(1), object(3)
memory usage: 119.0+ KB


In [36]:
dividends_df['sector'].unique()

array(['Real Estate', 'Consumer Cyclical', 'Financial Services',
       'Communication Services', 'Basic Materials', 'Industrials',
       'Technology', 'Utilities', nan, 'Consumer Defensive', 'Healthcare',
       'Energy'], dtype=object)

In [37]:
dividends_df['industry'].unique()

array(['Real Estate - Development', 'Department Stores',
       'Banks - Diversified', 'Telecom Services', 'Steel',
       'Infrastructure Operations', 'REIT - Healthcare Facilities',
       'Software - Application', 'Utilities - Diversified', nan,
       'Insurance - Property & Casualty', 'Auto & Truck Dealerships',
       'Recreational Vehicles', 'Farm Products',
       'Medical Care Facilities', 'Marine Shipping',
       'Real Estate Services', 'Entertainment',
       'Specialty Industrial Machinery', 'Resorts & Casinos',
       'Packaged Foods', 'Restaurants', 'Electrical Equipment & Parts',
       'Specialty Business Services', 'REIT - Industrial',
       'REIT - Diversified', 'Financial Data & Stock Exchanges',
       'Utilities - Regulated Electric', 'Building Products & Equipment',
       'Integrated Freight & Logistics', 'Utilities - Renewable',
       'Luxury Goods', 'Internet Retail', 'Lodging',
       'Apparel Manufacturing', 'REIT - Retail', 'Conglomerates',
       'Appare

In [38]:
companies_without_sector = dividends_df[dividends_df['sector'].isna()]['company'].unique()
print(f"The following companies do not have sector information: {companies_without_sector}.\n")

companies_without_industry = dividends_df[dividends_df['industry'].isna()]['company'].unique()
print(f"The following companies do not have industry information: {companies_without_industry}.")

The following companies do not have sector information: ['Smart US Large Value ETF' 'Smart US Small Cap ETF'
 'Smart US Mid Cap ETF' 'Smart US 500 (NZD Hedged) ETF'
 'Smart US Large Growth ETF' 'Smart US 500 ETF' 'Smart US ESG ETF'
 'Smart Total World (NZD Hedged) ETF' 'Smart Total World ETF'
 'Smart NZ Top 10 ETF' 'Templeton Emerging Markets Investment Trust Plc'
 'Private Land and Property Fund' 'Smart Australian Top 20 ETF'
 'Smart S&P/NZX 50 ETF' 'Smart NZ Cash ETF' 'Smart NZ Bond ETF'
 'Smart NZ Property ETF' 'Smart S&P/NZX NZ Government Bond ETF'
 'Smart Australian Mid Cap ETF' 'Marlin Global Limited'
 'Smart NZ Mid Cap ETF' 'Smart Healthcare Innovation ETF'
 'Kingfish Limited' 'Smart Japan ESG ETF'
 'Smart Global Infrastructure ETF' 'Henderson Far East Income Limited'
 'Smart Global Property ETF' 'Smart Global Government Bond ETF'
 'Smart Global Bond ETF' 'Smart NZ Top 50 ETF' 'F&C Investment Trust PLC'
 'Smart Europe ESG ETF' 'Smart Europe ETF' 'Smart Global ESG ETF'
 'Smart Em

In [39]:
companies_without_sector == companies_without_industry

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [40]:
sector_dict = {
    'Templeton Emerging Markets Investment Trust Plc': 'Financial Services',
    'Private Land and Property Fund':'Real Estate',
    'Marlin Global Limited': 'Financial Services',
    'Kingfish Limited': 'Financial Services',
    'Henderson Far East Income Limited': 'Financial Services',
    'F&C Investment Trust PLC': 'Financial Services',
    'Downer EDI Limited': 'Industrials',
    'Barramundi Limited': 'Financial Services',
    'The Bankers Investment Trust Plc':'Financial Services'
}

industry_dict = {
    'Templeton Emerging Markets Investment Trust Plc': 'Asset Management',
    'Private Land and Property Fund': 'Real Estate - Speciality',
    'Marlin Global Limited': 'Asset Management',
    'Kingfish Limited': 'Asset Management',
    'Henderson Far East Income Limited': 'Asset Management',
    'F&C Investment Trust PLC': 'Asset Management',
    'Downer EDI Limited': 'Engineering & Construction',
    'Barramundi Limited': 'Asset Management',
    'The Bankers Investment Trust Plc': 'Asset Management'
}

In [41]:
dividends_df['sector'] = dividends_df['company'].map(sector_dict).fillna(dividends_df['sector'])
dividends_df['industry'] = dividends_df['company'].map(industry_dict).fillna(dividends_df['industry'])

In [42]:
dividends_df['dividend_yield'] = dividends_df['dividends'].div(dividends_df['close'], axis=0)

In [43]:
non_etf_dividends_df = dividends_df[~dividends_df['company'].str.contains('ETF')]
non_etf_dividends_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1738 entries, 2022-08-30 00:00:00+12:00 to 2024-11-21 00:00:00+13:00
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   close           1738 non-null   float64
 1   dividends       1738 non-null   float64
 2   company         1738 non-null   object 
 3   sector          1738 non-null   object 
 4   industry        1738 non-null   object 
 5   year            1738 non-null   int32  
 6   dividend_yield  1738 non-null   float64
dtypes: float64(3), int32(1), object(3)
memory usage: 101.8+ KB


In [44]:
non_etf_dividends_df.head()

,close,dividends,company,sector,industry,year,dividend_yield
Date,,,,,,,
2022-08-30 00:00:00+12:00,2.541574,0.010700,Winton Land Limited (NS),Real Estate,Real Estate - Development,2022,0.004210
2023-02-28 00:00:00+13:00,1.916445,0.020600,Winton Land Limited (NS),Real Estate,Real Estate - Development,2023,0.010749
2023-08-28 00:00:00+12:00,2.342819,0.021600,Winton Land Limited (NS),Real Estate,Real Estate - Development,2023,0.009220
2024-02-26 00:00:00+13:00,2.500000,0.007639,Winton Land Limited (NS),Real Estate,Real Estate - Development,2024,0.003056
2015-11-25 00:00:00+13:00,1.487685,0.050000,The Warehouse Group Limited,Consumer Cyclical,Department Stores,2015,0.033609


In [45]:
non_etf_dividends_df.groupby('sector')['company'].nunique()

sector
Basic Materials            3
Communication Services     5
Consumer Cyclical         14
Consumer Defensive        13
Energy                     1
Financial Services        14
Healthcare                 9
Industrials               18
Real Estate               13
Technology                 4
Utilities                  7
Name: company, dtype: int64

In [46]:
non_etf_dividends_df[non_etf_dividends_df['sector'] == 'Basic Materials']

,close,dividends,company,sector,industry,year,dividend_yield
Date,,,,,,,
2022-03-10 00:00:00+13:00,7.820786,0.323529,Vulcan Steel Limited,Basic Materials,Steel,2022,0.041368
2022-09-20 00:00:00+12:00,7.013064,0.427900,Vulcan Steel Limited,Basic Materials,Steel,2022,0.061015
2022-09-22 00:00:00+12:00,7.465656,0.375000,Vulcan Steel Limited,Basic Materials,Steel,2022,0.050230
2023-03-09 00:00:00+13:00,8.245689,0.288235,Vulcan Steel Limited,Basic Materials,Steel,2023,0.034956
2023-09-27 00:00:00+13:00,7.758816,0.328682,Vulcan Steel Limited,Basic Materials,Steel,2023,0.042362
2024-02-29 00:00:00+13:00,8.150040,0.141176,Vulcan Steel Limited,Basic Materials,Steel,2024,0.017322
2024-09-26 00:00:00+12:00,8.205516,0.126400,Vulcan Steel Limited,Basic Materials,Steel,2024,0.015404
2025-03-13 00:00:00+13:00,8.770000,0.025882,Vulcan Steel Limited,Basic Materials,Steel,2025,0.002951
2015-09-23 00:00:00+12:00,1.447967,0.017963,Steel & Tube Holdings Limited,Basic Materials,Steel,2015,0.012406


In [47]:
non_etf_dividends_df[non_etf_dividends_df['sector'] == 'Energy']

,close,dividends,company,sector,industry,year,dividend_yield
Date,,,,,,,
2016-03-09 00:00:00+13:00,2.943734,0.198458,Channel Infrastructure NZ Limited,Energy,Oil & Gas Refining & Marketing,2016,0.067417
2018-09-12 00:00:00+12:00,2.299199,0.029769,Channel Infrastructure NZ Limited,Energy,Oil & Gas Refining & Marketing,2018,0.012948
2019-03-06 00:00:00+13:00,1.878878,0.044653,Channel Infrastructure NZ Limited,Energy,Oil & Gas Refining & Marketing,2019,0.023766
2019-09-11 00:00:00+12:00,1.980632,0.019846,Channel Infrastructure NZ Limited,Energy,Oil & Gas Refining & Marketing,2019,0.010020
2023-03-09 00:00:00+13:00,1.392000,0.049615,Channel Infrastructure NZ Limited,Energy,Oil & Gas Refining & Marketing,2023,0.035643
2025-03-12 00:00:00+13:00,1.950000,0.066000,Channel Infrastructure NZ Limited,Energy,Oil & Gas Refining & Marketing,2025,0.033846


In [48]:
non_etf_dividends_df[non_etf_dividends_df['sector'] == 'Technology']

,close,dividends,company,sector,industry,year,dividend_yield
Date,,,,,,,
2017-03-09 00:00:00+13:00,2.583124,0.021977,Vista Group International Limited,Technology,Software - Application,2017,0.008508
2017-09-07 00:00:00+12:00,2.538454,0.011441,Vista Group International Limited,Technology,Software - Application,2017,0.004507
2018-03-09 00:00:00+13:00,2.545461,0.016561,Vista Group International Limited,Technology,Software - Application,2018,0.006506
2018-09-12 00:00:00+12:00,3.776473,0.015255,Vista Group International Limited,Technology,Software - Application,2018,0.004039
2019-03-08 00:00:00+13:00,4.467782,0.020022,Vista Group International Limited,Technology,Software - Application,2019,0.004481
2019-09-12 00:00:00+12:00,3.804246,0.011441,Vista Group International Limited,Technology,Software - Application,2019,0.003007
2015-09-23 00:00:00+12:00,0.759545,0.015000,Solution Dynamics Limited,Technology,Software - Application,2015,0.019749
2016-02-26 00:00:00+13:00,1.194930,0.030000,Solution Dynamics Limited,Technology,Software - Application,2016,0.025106
2016-08-22 00:00:00+12:00,1.674811,0.022500,Solution Dynamics Limited,Technology,Software - Application,2016,0.013434


In [49]:
non_etf_dividends_df = non_etf_dividends_df[~non_etf_dividends_df['sector'].isin(['Energy', 'Basic Materials'])]

In [50]:
annual_dividend_yield = non_etf_dividends_df.groupby(['company', 'year']).apply(
    lambda g: pd.Series({
        'annual_dividend': g['dividends'].sum(),
        'avg_price': g['close'].mean(),
        'sector': g['sector'].iloc[0]
    })
).reset_index()

# Compute annualized dividend yield
annual_dividend_yield['annual_yield'] = annual_dividend_yield['annual_dividend'] / annual_dividend_yield['avg_price']
annual_dividend_yield

/var/folders/gq/xq2cwt6563vcdxlq8xdfw5hw0000gn/T/ipykernel_4689/2752375087.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  annual_dividend_yield = non_etf_dividends_df.groupby(['company', 'year']).apply(


,company,year,annual_dividend,avg_price,sector,annual_yield
0,2 Cheap Cars Group Limited,2023,0.041600,0.769250,Consumer Cyclical,0.054079
1,2 Cheap Cars Group Limited,2024,0.074823,0.831586,Consumer Cyclical,0.089976
2,AFT Pharmaceuticals Limited,2023,0.011000,3.471987,Healthcare,0.003168
3,AFT Pharmaceuticals Limited,2024,0.016000,3.100000,Healthcare,0.005161
4,ANZ Group Holdings Limited,2015,1.942656,19.302325,Financial Services,0.100644
...,...,...,...,...,...,...
795,Westpac Banking Corporation,2023,1.538286,21.356007,Financial Services,0.072031
796,Westpac Banking Corporation,2024,1.635783,31.486190,Financial Services,0.051952
797,Winton Land Limited (NS),2022,0.010700,2.541574,Real Estate,0.004210
798,Winton Land Limited (NS),2023,0.042200,2.129632,Real Estate,0.019816


In [51]:
# Step 3: Identify outliers per sector-year
def detect_outliers(subdf):
    # Use IQR method (can be replaced with z-score if preferred)
    q1 = subdf['annual_yield'].quantile(0.25)
    q3 = subdf['annual_yield'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 2 * iqr
    upper = q3 + 2 * iqr
    mask = (subdf['annual_yield'] < lower) | (subdf['annual_yield'] > upper)
    outliers = subdf[mask]
    clean = subdf[~mask]
    return clean, outliers

In [52]:

clean_list = []
outlier_list = []
for (sector, year), group in annual_dividend_yield.groupby(['sector', 'year']):
    clean, outliers = detect_outliers(group)
    if not clean.empty:
        clean_list.append(clean)
    if not outliers.empty:
        outlier_list.append(outliers)

# Combine the results
clean_df = pd.concat(clean_list).reset_index(drop=True)
outliers_df = pd.concat(outlier_list).reset_index(drop=True)

In [55]:
outliers_df

,company,year,annual_dividend,avg_price,sector,annual_yield
0,Hallenstein Glasson Holdings Limited,2016,0.300000,1.512193,Consumer Cyclical,0.198387
1,Hallenstein Glasson Holdings Limited,2017,0.315000,1.916094,Consumer Cyclical,0.164397
2,Hallenstein Glasson Holdings Limited,2018,0.440000,2.943387,Consumer Cyclical,0.149488
3,Millennium & Copthorne Hotels New Zealand Limited,2022,0.070000,2.317082,Consumer Cyclical,0.030210
4,My Food Bag Group Limited,2022,0.085556,0.579993,Consumer Cyclical,0.147512
5,Restaurant Brands New Zealand Limited,2022,0.320000,12.668883,Consumer Cyclical,0.025259
6,Burger Fuel Group Limited,2024,0.270000,0.350000,Consumer Cyclical,0.771429
7,Hallenstein Glasson Holdings Limited,2025,0.245000,7.660000,Consumer Cyclical,0.031984
8,Tourism Holdings Limited,2025,0.025000,1.720000,Consumer Cyclical,0.014535
9,Fonterra Shareholders' Fund (NS),2016,0.400000,2.448684,Consumer Defensive,0.163353


In [60]:
dividends_df[dividends_df['company'] == 'The Bankers Investment Trust Plc']

,close,dividends,company,sector,industry,year,dividend_yield
Date,,,,,,,
2015-07-22 00:00:00+12:00,0.375007,0.091423,The Bankers Investment Trust Plc,Financial Services,Asset Management,2015,0.243790
2015-10-21 00:00:00+13:00,0.337434,0.091608,The Bankers Investment Trust Plc,Financial Services,Asset Management,2015,0.271484
2016-01-27 00:00:00+13:00,0.351616,0.088380,The Bankers Investment Trust Plc,Financial Services,Asset Management,2016,0.251354
2016-04-14 00:00:00+12:00,0.361132,0.082277,The Bankers Investment Trust Plc,Financial Services,Asset Management,2016,0.227831
2016-07-28 00:00:00+12:00,0.381832,0.074475,The Bankers Investment Trust Plc,Financial Services,Asset Management,2016,0.195046
2016-10-27 00:00:00+13:00,0.402781,0.075221,The Bankers Investment Trust Plc,Financial Services,Asset Management,2016,0.186754
2017-01-26 00:00:00+13:00,0.447546,0.079877,The Bankers Investment Trust Plc,Financial Services,Asset Management,2017,0.178478
2017-04-20 00:00:00+12:00,0.527746,0.080439,The Bankers Investment Trust Plc,Financial Services,Asset Management,2017,0.152420
2017-07-27 00:00:00+12:00,0.583817,0.082582,The Bankers Investment Trust Plc,Financial Services,Asset Management,2017,0.141452
